In [246]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio

# Neural Networks

In [247]:
path = 'ex4data1.mat'
data = sio.loadmat(path)
X = data['X']
y = data['y']
rows = X.shape[0]
params = X.shape[1]
weight = sio.loadmat('ex4weights.mat')
theta1 = weight['Theta1']
theta2 = weight['Theta2']
y.shape

(5000, 1)

In [248]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [249]:
def sigmoidGradient(z):
    return np.multiply(z, (1 - z))


In [250]:
def cost(theta1, theta2, X, y, num_label, alpha):
    rows = X.shape[0]
    params = X.shape[1]
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    X = np.matrix(X)
    theta1 = np.matrix(theta1)
    theta2 = np.matrix(theta2)
    A = sigmoid(X * theta1.T)
    A = np.insert(A, 0, values=np.ones(rows), axis=1)
    A = np.matrix(A)
    
    New = np.zeros((y.shape[0], 10))
    for i in range(rows):
        New[i, y[i] - 1] = 1
    y = New
    # print(A.shape, X.shape)
    # first = np.multiply(-y, np.log(sigmoid(X * theta1.T))) - np.multiply((1 - y), np.log(1 - sigmoid(X * theta1.T)))
    second =np.multiply(-y, np.log(sigmoid(A * theta2.T))) - np.multiply((1 - y), np.log(1 - sigmoid(A * theta2.T)))

    reg = np.sum(np.power(theta2, 2)) * (alpha / (2 * len(A)))
    
    return (np.sum(second)) / len(X) + reg


In [251]:
C = cost(theta1, theta2, X, y, 25, 1)
C

0.3449633357692933

# Backpropagation

In [252]:
sigmoidGradient(0)

0

In [253]:
epsilon_init = 0.12
alpha = 1
L_2 = 25
L_3 = 10
theta1 = np.matrix(np.random.rand(L_2, X.shape[1] + 1) * (2 * epsilon_init) - epsilon_init)
theta2 = np.matrix(np.random.rand(L_3, L_2 + 1) * (2 * epsilon_init) - epsilon_init)
theta2.shape, theta1.shape


((10, 26), (25, 401))

In [254]:
def forwardPropagation(A, theta):
    return (A.T * theta.T).T
def insertfunction(A):
    A = np.insert(A, 0, 1, axis=0)
    return np.matrix(A)
# a_1 = np.matrix(X)
# z_2 = forwardPropagation(a_1, theta1)
# a_2 = sigmoid(z_2)
# z_3 = forwardPropagation(a_2, theta2)
# a_3 = sigmoid(z_3)
# z_2.shape

In [255]:
# New = np.zeros((y.shape[0], 10))
# for i in range(rows):
#     New[i, y[i] - 1] = 1
# y = New
# delta3 = a_3 - y
# y.shape, delta3.shape

In [256]:
def BPfunction(X, y, theta1, theta2):
    
    for i in range(len(X)):
        New = np.zeros((10, 1))
        a1 = X[i, :].T
        New[y[i] - 1] = 1
        y_out = New # .T只是暂时转置，真正转置还是要用reshape

        #先转换成列向量！
        a1 = insertfunction(a1)
        # print(a1.shape)
        a1 = np.matrix(a1)
        z2 = forwardPropagation(a1.T, theta1) 
        # print(z2.shape)
        a2 = sigmoid(z2)
        a2 = insertfunction(a2)
        z3 = forwardPropagation(a2, theta2)
        a3 = sigmoid(z3)

        delta3 = a3 - y_out
        # print(delta3.shape)
        delta2 = np.multiply(theta2.T * delta3, sigmoidGradient(a2))
        # delta2 = np.multiply(theta2.T, delta3.T) * sigmoidGradient(z2)
        delta2 = delta2[1 :delta2.shape[0], :]

        delta_l2 = delta_l2 + np.multiply(delta3, a2.T)
        delta_l1 = delta_l1 + np.multiply(delta2, a1.T)

    return np.sum(delta_l1) / len(X), np.sum(delta_l2) / len(X)

In [257]:
def gradient(delta, len, alpha, theta):
    params = theta.shape[1]
    grad = np.zeros((params, 1))
    for i in range(1, params):
        grad[i, :] = (delta / len) +  (alpha * np.sum(theta))
    grad[0, :] = delta / len

    return grad
    

In [258]:
delta_l1, delta_l2 = BPfunction(X, y, theta1, theta2)
grad = gradient(delta_l2, rows, alpha, theta2)

UnboundLocalError: local variable 'delta_l2' referenced before assignment